# I. PEMAHAMAN DOMAIN DAN TUJUAN

## 1.1. Pemahaman Penyakit Ginjal Kronis

Penyakit Ginjal Kronis (PGK) adalah suatu proses patofisiologis dengan etiologi yang beragam, mengakibatkan penurunan fungsi ginjal yang progresif, penurunan fungsi ini bersifat kronis dan irreversible.

Data didapatkan dari UCI Machine Learning Repository. Berisikan 400 data dan memiliki 25 atribut, terdiri dari 1 kelas target dan 24 atribut. 

## 1.2. Tujuan

Identifikasi PGK dengan atribut yang paling berpengaruh

# II. PEMBUATAN DATASET DAN TARGET

## 2.1 Data UCI

Data didapat dari: https://archive.ics.uci.edu/ml/datasets/chronic_kidney_disease

Data tersebut di-import dalam mariaDB

In [1]:
import numpy as np
import pandas as pd
import pymysql as pskl
import matplotlib.pyplot as plt

# Menampilkan semua array
# np.set_printoptions(threshold=np.nan)

In [2]:
def dataset(retrieve="numeric", id=0, target=0): #hasil bertipe dataframe
    """
    parameters
    ----------
    retrieve: (all, numeric, polinom)
        all    : semua kolom
        numeric: hanya kolom numeric
        polinom: hanya kolom polinom
    id: (0, 1)
        0: tanpa kolom id
        1: dengan kolom id
    class: (0, 1)
        0: tanpa kolom class
        1: dengan kolom class
    
    """
    connection = pskl.connect(host="localhost", user="root", passwd="", database="knn")
    cursor = connection.cursor()
    if(retrieve=="all"):
        cols = ["id", "age", "bp", "sg", "al", "su", "bgr", "bu", "sc", "sod", "pot", "hemo", "pcv", "wbcc", "rbcc", "rbc", "pc", "pcc", "ba", "htn", "dm", "cad", "appet", "pe", "ane", "class"]
        retrieve = "SELECT id, age, bp, sg, al, su, bgr, bu, sc, sod, pot, hemo, pcv, wbcc, rbcc, rbc, pc, pcc, ba, htn, dm, cad, appet, pe, ane, class FROM ckd_preprocessing3"
    elif(retrieve=="numeric"):
        cols = ["id", "age", "bp", "sg", "al", "su", "bgr", "bu", "sc", "sod", "pot", "hemo", "pcv", "wbcc", "rbcc", "class"]
        retrieve = "SELECT id, age, bp, sg, al, su, bgr, bu, sc, sod, pot, hemo, pcv, wbcc, rbcc, class FROM ckd_preprocessing3"
    elif(retrieve=="polinom"):
        cols = ["id", "rbc", "pc", "pcc", "ba", "htn", "dm", "cad", "appet", "pe", "ane", "class"]
        retrieve = "SELECT id, rbc, pc, pcc, ba, htn, dm, cad, appet, pe, ane, class FROM ckd_preprocessing3"
    resolveall = cursor.execute(retrieve)
    rows_tupple = cursor.fetchall()
    data = pd.DataFrame(list(rows_tupple))
    data.columns = cols
    if(id==0):
        data = data.drop(["id"], axis=1)
    if(target==0):
        data = data.drop(["class"], axis=1)
    return data
dataset(retrieve="polinom").head(5)

,rbc,pc,pcc,ba,htn,dm,cad,appet,pe,ane
0,None,normal,notpresent,notpresent,yes,yes,no,good,no,no
1,None,normal,notpresent,notpresent,no,no,no,good,no,no
2,normal,normal,notpresent,notpresent,no,yes,no,poor,no,yes
3,normal,abnormal,present,notpresent,yes,no,no,poor,yes,yes
4,normal,normal,notpresent,notpresent,no,no,no,good,no,no


## 2.2. Menentukan Atribut Target

Atribut "class" sebagai target dataset

# III. DATA CLEANING DAN PREPROCESSING

referensi: http://digilib.unila.ac.id/20585/4/II.%20TINJAUAN%20PUSTAKA.pdf

In [3]:
df = dataset()
df.head()

,age,bp,sg,al,su,bgr,bu,sc,sod,pot,hemo,pcv,wbcc,rbcc
0,48.0,80.0,1.020,1.0,0.0,121.0,36.0,1.2,NaN,NaN,15.4,44.0,7800.0,5.2
1,7.0,50.0,1.020,4.0,0.0,NaN,18.0,0.8,NaN,NaN,11.3,38.0,6000.0,NaN
2,62.0,80.0,1.010,2.0,3.0,423.0,53.0,1.8,NaN,NaN,9.6,31.0,7500.0,NaN
3,48.0,70.0,1.005,4.0,0.0,117.0,56.0,3.8,111.0,2.5,11.2,32.0,6700.0,3.9
4,51.0,80.0,1.010,2.0,0.0,106.0,26.0,1.4,NaN,NaN,11.6,35.0,7300.0,4.6


In [4]:
k1 = 0.25
k3 = 0.75

kuantil = df.quantile([k1,k3])
kuantil

,age,bp,sg,al,su,bgr,bu,sc,sod,pot,hemo,pcv,wbcc,rbcc
0.25,42.0,70.0,1.01,0.0,0.0,99.0,27.0,0.9,135.0,3.8,10.3,32.0,6500.0,3.9
0.75,64.5,80.0,1.02,2.0,0.0,163.0,66.0,2.8,142.0,4.9,15.0,45.0,9800.0,5.4


In [5]:
len(df.columns)

14

In [6]:
for i in range(len(df.columns)):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13


In [7]:
df.iloc[:,0].quantile([k3]).values - df.iloc[:,0].quantile([k1]).values

array([ 22.5])

In [8]:
filtering_rule = df.apply(lambda x: print(x.name), axis=0)

age
bp
sg
al
su
bgr
bu
sc
sod
pot
hemo
pcv
wbcc
rbcc


In [9]:
df["age"].head()

0    48.0
1     7.0
2    62.0
3    48.0
4    51.0
Name: age, dtype: float64

In [10]:
filtering_rule = df.apply(lambda x: df[x.name], axis=0)
filtering_rule.head()

,age,bp,sg,al,su,bgr,bu,sc,sod,pot,hemo,pcv,wbcc,rbcc
0,48.0,80.0,1.020,1.0,0.0,121.0,36.0,1.2,NaN,NaN,15.4,44.0,7800.0,5.2
1,7.0,50.0,1.020,4.0,0.0,NaN,18.0,0.8,NaN,NaN,11.3,38.0,6000.0,NaN
2,62.0,80.0,1.010,2.0,3.0,423.0,53.0,1.8,NaN,NaN,9.6,31.0,7500.0,NaN
3,48.0,70.0,1.005,4.0,0.0,117.0,56.0,3.8,111.0,2.5,11.2,32.0,6700.0,3.9
4,51.0,80.0,1.010,2.0,0.0,106.0,26.0,1.4,NaN,NaN,11.6,35.0,7300.0,4.6


In [11]:
df.age[np.mean(df.age)<df.age].head() #filter ambil yang lebih dari rata2

2    62.0
5    60.0
6    68.0
8    52.0
9    53.0
Name: age, dtype: float64

In [12]:
df.apply(lambda x: df[x.name].quantile(k1), axis=0)

age       42.00
bp        70.00
sg         1.01
al         0.00
su         0.00
bgr       99.00
bu        27.00
sc         0.90
sod      135.00
pot        3.80
hemo      10.30
pcv       32.00
wbcc    6500.00
rbcc       3.90
dtype: float64

In [13]:
df.apply(lambda x: df[x.name].quantile(k3), axis=0)

age       64.50
bp        80.00
sg         1.02
al         2.00
su         0.00
bgr      163.00
bu        66.00
sc         2.80
sod      142.00
pot        4.90
hemo      15.00
pcv       45.00
wbcc    9800.00
rbcc       5.40
dtype: float64

In [14]:
df.apply(lambda x: (x < df[x.name].quantile(k1)), axis=0).head()

,age,bp,sg,al,su,bgr,bu,sc,sod,pot,hemo,pcv,wbcc,rbcc
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,True,True,False,False,False,False,True,True,False,False,False,False,True,False
2,False,False,False,False,False,False,False,False,False,False,True,True,False,False
3,False,False,True,False,False,False,False,False,True,True,False,False,False,False
4,False,False,False,False,False,False,True,False,False,False,False,False,False,False


In [15]:
filtering = df.apply(lambda x: (x < df[x.name].quantile(k1)-((df[x.name].quantile(k3)-df[x.name].quantile(k1))*1.5))|(x > df[x.name].quantile(k3)+((df[x.name].quantile(k3)-df[x.name].quantile(k1))*1.5)), axis=0)

In [16]:
filtering

,age,bp,sg,al,su,bgr,bu,sc,sod,pot,hemo,pcv,wbcc,rbcc
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,True,True,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,True,True,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,True,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,True,True,False,False,False,False,False
7,False,False,False,False,True,True,False,False,False,False,False,False,False,False
8,False,True,False,False,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,True,True,False,False,False,False,False


In [17]:
filtered_df = df[~(filtering).any(axis=1)]
filtered_df

,age,bp,sg,al,su,bgr,bu,sc,sod,pot,hemo,pcv,wbcc,rbcc
0,48.0,80.0,1.020,1.0,0.0,121.0,36.0,1.20,NaN,NaN,15.4,44.0,7800.0,5.2
4,51.0,80.0,1.010,2.0,0.0,106.0,26.0,1.40,NaN,NaN,11.6,35.0,7300.0,4.6
5,60.0,90.0,1.015,3.0,0.0,74.0,25.0,1.10,142.0,3.2,12.2,39.0,7800.0,4.4
13,68.0,70.0,NaN,NaN,NaN,98.0,86.0,4.60,135.0,3.4,9.8,NaN,NaN,NaN
16,47.0,70.0,1.015,2.0,0.0,99.0,46.0,2.20,138.0,4.1,12.6,NaN,NaN,NaN
17,47.0,80.0,NaN,NaN,NaN,114.0,87.0,5.20,139.0,3.7,12.1,NaN,NaN,NaN
19,62.0,60.0,1.015,1.0,0.0,100.0,31.0,1.60,NaN,NaN,10.3,30.0,5300.0,3.7
23,21.0,70.0,1.010,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,61.0,60.0,1.025,0.0,0.0,108.0,75.0,1.90,141.0,5.2,9.9,29.0,8400.0,3.7
26,75.0,80.0,1.015,0.0,0.0,156.0,45.0,2.40,140.0,3.4,11.6,35.0,10300.0,4.0


In [18]:
def outliers_removing(df, k1=0.25, k3=0.75):
    """
    parameters:
    -----------
    df: tipe dataframe, hanya menerima numeric
    
    mendeteksi data yang diluar batas bawah dan batas atas
    batas bawah = k1 - (k3-k1)*1.5
    batas atas = k3 + (k3-k1)*1.5
    """
    filtering = df.apply(lambda x: (x < df[x.name].quantile(k1)-((df[x.name].quantile(k3)-df[x.name].quantile(k1))*1.5))|(x > df[x.name].quantile(k3)+((df[x.name].quantile(k3)-df[x.name].quantile(k1))*1.5)), axis=0)
    filtered_df = df[~(filtering).any(axis=1)]
    return filtered_df

In [20]:
df = dataset()
filtered_df = outliers_removing(df)
filtered_df

,age,bp,sg,al,su,bgr,bu,sc,sod,pot,hemo,pcv,wbcc,rbcc
0,48.0,80.0,1.020,1.0,0.0,121.0,36.0,1.20,NaN,NaN,15.4,44.0,7800.0,5.2
4,51.0,80.0,1.010,2.0,0.0,106.0,26.0,1.40,NaN,NaN,11.6,35.0,7300.0,4.6
5,60.0,90.0,1.015,3.0,0.0,74.0,25.0,1.10,142.0,3.2,12.2,39.0,7800.0,4.4
13,68.0,70.0,NaN,NaN,NaN,98.0,86.0,4.60,135.0,3.4,9.8,NaN,NaN,NaN
16,47.0,70.0,1.015,2.0,0.0,99.0,46.0,2.20,138.0,4.1,12.6,NaN,NaN,NaN
17,47.0,80.0,NaN,NaN,NaN,114.0,87.0,5.20,139.0,3.7,12.1,NaN,NaN,NaN
19,62.0,60.0,1.015,1.0,0.0,100.0,31.0,1.60,NaN,NaN,10.3,30.0,5300.0,3.7
23,21.0,70.0,1.010,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,61.0,60.0,1.025,0.0,0.0,108.0,75.0,1.90,141.0,5.2,9.9,29.0,8400.0,3.7
26,75.0,80.0,1.015,0.0,0.0,156.0,45.0,2.40,140.0,3.4,11.6,35.0,10300.0,4.0


In [22]:
filtered_df.to_csv("CKD_filtered_df.csv")